# 🦌 Elk Model Explorer ⚡

The [Elk Model Repositoy](https://github.com/eclipse/elk-models) was converted to elk json and included as a set of fixtures to test against. This notebook loads those fixtures for users to explore to understand what kinds of layout variety possible as well and see the originating elk json.

In [1]:
import json
import asyncio

from pathlib import Path
from IPython.display import display, JSON
from ipyelk import ElkDiagram, tests
from ipyelk.diagram.elk_text_sizer import size_labels, ElkTextSizer
from ipyelk.diagram.elk_model import ElkLabel


In [3]:
ran = []
async def size_labels(text_sizer, labels):
    ran.append(labels)
    sizes = await text_sizer.measure(tuple(ElkLabel.from_dict(l) for l in labels))
    
    for size, label in zip(sizes, labels):
        label["width"] = size.width
        label["height"] = size.height
    
    
def collect_labels(data):
    
    labels = []
    labels += data.get("labels", [])
    for prop in ["ports", "children", "edges"]:
        for value in data.get(prop, []):
            labels += collect_labels(value)
    return labels

def default_node_size(data, width=40, height=40):
    if "width" not in data:
        data["width"] = width
    if "height" not in data:
        data["height"] = height
    for child in data.get("children", []):
        default_node_size(child)

def fixture_explorer():
    #TODO some labels do not have a width / height provided ... could use the text sizer widget and size labels accordingly
    #TODO some nodes do not have a specified width / height ... set a default
    fixtures = Path(tests.__file__).parent / "fixtures"
    fixtures

    options = [f.relative_to(fixtures) for f in fixtures.rglob("*.json")]
    len(options)
    import ipywidgets as W
    diagram = ElkDiagram(layout={"flex":"1"})
    selector = W.Dropdown(options=options)
    err_btn = W.Button()
    output = W.Output()
    text_sizer = ElkTextSizer(max_size=20)

    def set_err(message):
        if message:
            err_btn.icon = "warning"
            err_btn.tooltip = message
        else:
            err_btn.icon = "check"
            err_btn.tooltip = ""

    
    async def _load():
        # load fixture elk json
        data = json.loads((fixtures / selector.value).read_text())
        
        # add width and height to labels
        await size_labels(text_sizer, collect_labels(data))
        
        # add default size to nodes
        default_node_size(data)
        
        try:
            diagram.value = {"id":"root"}  # needed for now to clear sprotty diagram. until fixed https://github.com/jupyrdf/ipyelk/issues/17
            diagram.value = data
            set_err("")
        except Exception as e:
            diagram.value = {"id":"root"}
            set_err(str(e))
            
    def update(change=None):
        asyncio.create_task(_load())
        
    def refresh_json_view(change=None):
        output.clear_output()
        with output:
            display(JSON(diagram.value))
    
            
    diagram.observe(refresh_json_view, "value")


    model_explorer = W.VBox(children=[

        W.HBox(
            children=[
                selector,
                err_btn,
            ]
        ),
        W.HBox(
            children=[
                diagram,
                output,
            ],
            layout={"flex":"1"})

    ],
        layout={"height":"100%"}  
    )
    selector.observe(update, "value")
    update()
    return model_explorer, diagram, output, refresh_json_view

if __name__ == "__main__":
    explorer, diagram, output, refresh_json_view = fixture_explorer()
    display(explorer)

In [32]:
# TODO understand why this refresh_json_view function works if called manually but no longer seems to update the output view using the observers

In [6]:
refresh_json_view()